Goal : Preserving conversational history and generating summary at the end of converstaion

Step 1: importing required packages

In [ ]:
! pip install  langgraph
! pip install langchain 
! pip install langchain_core_google_genai 
! pip install typing_extensions 
! pip install langchain_core


In [ ]:
from langgraph.graph import START, END, StateGraph, add_messages , MessagesState
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Literal, Annotated
from typing_extensions import TypedDict 
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage ,RemoveMessage,SystemMessage
from _collections_abc import Sequence
from langgraph.checkpoint.memory import InMemorySaver

Step 2: Define State

In [ ]:
class State(MessagesState):
    summary:str

Step 3: Define node

In [ ]:
chat = ChatGoogleGenerativeAI(model="gemini-flash-latest", temperature=0, seed  =365)


In [ ]:
def ask_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_question:")
    
    question = "What is your question?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

In [ ]:
def chatbot(state: State) -> State:
    
    print(f"\n-------> ENTERING chatbot:")
    for i in state["messages"]:
        i.pretty_print()
    
    system_message=f"here's quick summary of what's been disscuss so far: {state.get('summary', '')} . keep this in mind as you answer the next question."
    response = chat.invoke([SystemMessage(content=system_message)] + state["messages"])
    response.pretty_print()
    
    return {"messages":[response]}

In [ ]:
def summarize_messages(state: State) -> State:
    print(f"\n-------> ENTERING trim_messages:")
    
    new_conversation = ""
    for i in state["messages"]:
        new_conversation += f"{i.type}: {i.content}\n\n"
        
    summary_instructions = f'''
Update the ongoing summary by incorporating the new lines of conversation below.  
Build upon the previous summary rather than repeating it so that the result  
reflects the most recent context and developments.
Previous Summary:
{state.get("summary", "")}

New Conversation:
{new_conversation}
'''
    
    print(summary_instructions)
    
    summary = chat.invoke([HumanMessage(summary_instructions)])
    
    remove_messages = [RemoveMessage(id = i.id) for i in state["messages"][:]]
    
    return {"messages":remove_messages, "summary":summary.content}

Step 4: define graph

In [ ]:

graph = StateGraph(State)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("summarize_messages", summarize_messages)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "summarize_messages")

In [ ]:
checkerpoint= InMemorySaver()
graph_compiled = graph.compile(checkerpoint)

In [ ]:
graph_compiled

In [ ]:
config1 = {"configurable": {"thread_id": "1"}}

Step 5: Test the graph

In [ ]:
graph_compiled.invoke({"messages": []},config1)


-------> ENTERING ask_question:
What is your question?

-------> ENTERING chatbot:
================================== Ai Message ==================================

What is your question?
================================ Human Message =================================

what was he known for/
================================== Ai Message ==================================

[{'type': 'text', 'text': 'I apologize, but the summary you mentioned was empty, so I don\'t know who "he" is.\n\nCould you please provide the name of the person we were discussing so I can tell you what he was known for?', 'extras': {'signature': 'CoMHAXLI2nzezAsLrSj/7XRgQIG6FYHuek3tYWtgjmLzfht8GMIsolSlWcUjPh3fYTnU0mE/XP4vBsRy0Ka+wPpzid8vukbAmn7XdiJE9qw19pA0EqiI3bHS6SW4gD1WY9Y+q/XozGXBZOExU37pJ1IGROJUCDU0Hdc56pGW7n4q9E7S32X9RjLks6Aip3ct3nX3YPJPJaYqOVdzm8Wr7E0l4Jc4WarzE3ymX/uIMRoXHl0fUwEOztj3vLglbqc5XnPMNsqX4ORSXmyTUWo5vh/oDF37Ik7MA1eGnIIL7Ku7HcE+mEU0X05mMS8Ffp1hT3L4LozddDGs7zz3ql6maZjOZvbdvn5d0+0fM0x4qjyNIF6S4Apr+nN

{'messages': [],
 'summary': 'The conversation began with the human asking what an unidentified person ("he") was known for. The AI was unable to answer this question due to a lack of context (noting that the previous summary was empty) and requested the human to provide the name of the individual.'}